# 阈值逻辑单元 (TLU) 与感知机网络

本教程介绍神经网络的基础构建单元——阈值逻辑单元(Threshold Logic Unit, TLU)，
以及基于TLU的感知机(Perceptron)分类器。

## 核心概念

**TLU** 是最简单的人工神经元模型，由Warren McCulloch和Walter Pitts于1943年提出。
其工作原理如下：

1. 接收多个输入信号 $x_1, x_2, ..., x_n$
2. 每个输入乘以对应权重 $w_1, w_2, ..., w_n$
3. 计算加权和 $z = \sum_{i=1}^{n} w_i x_i + b$
4. 通过阶跃函数(step function)输出：$y = \text{step}(z) = \begin{cases} 1 & z \geq 0 \\ 0 & z < 0 \end{cases}$

**感知机** 由Frank Rosenblatt于1957年发明，是单层TLU网络的具体实现。

## 1. 环境配置与数据准备

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# 设置随机种子确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2. 加载并探索鸢尾花数据集

In [ ]:
# 加载经典的鸢尾花数据集
iris = load_iris()

# 数据集基本信息
print("数据集特征名称:", iris.feature_names)
print("数据集类别名称:", iris.target_names)
print("数据集形状:", iris.data.shape)
print("标签分布:", np.bincount(iris.target))

In [ ]:
# 选取花瓣长度(petal length)和花瓣宽度(petal width)作为特征
# 这两个特征具有较强的区分能力
X = iris.data[:, [2, 3]]  # 第3、4列：花瓣长度、花瓣宽度

# 将多分类问题转换为二分类问题
# Iris-setosa (类别0) vs 非Iris-setosa (类别1、2)
y = (iris.target == 0).astype(int)

print("特征矩阵形状:", X.shape)
print("标签向量形状:", y.shape)
print("正类(Setosa)样本数:", np.sum(y == 1))
print("负类(非Setosa)样本数:", np.sum(y == 0))

## 3. 数据可视化

In [ ]:
# 可视化两类样本的分布
plt.figure(figsize=(8, 6))

# 绘制Setosa类
plt.scatter(X[y == 1, 0], X[y == 1, 1], 
            c='blue', marker='o', label='Iris-setosa', edgecolors='black')

# 绘制非Setosa类
plt.scatter(X[y == 0, 0], X[y == 0, 1], 
            c='red', marker='s', label='Non-setosa', edgecolors='black')

plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.title('Iris Dataset: Setosa vs Non-Setosa')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 训练感知机分类器

In [ ]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

print("训练集大小:", X_train.shape[0])
print("测试集大小:", X_test.shape[0])

In [ ]:
# 创建并训练感知机分类器
# max_iter: 最大迭代次数
# eta0: 学习率
# tol: 收敛阈值
# random_state: 随机种子
perceptron = Perceptron(
    max_iter=1000,
    eta0=0.1,
    tol=1e-3,
    random_state=RANDOM_SEED
)

# 训练模型
perceptron.fit(X_train, y_train)

print("训练完成")
print("权重向量:", perceptron.coef_)
print("偏置项:", perceptron.intercept_)

## 5. 模型评估

In [ ]:
# 在测试集上进行预测
y_pred = perceptron.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"测试集准确率: {accuracy:.4f}")

# 详细分类报告
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=['Non-Setosa', 'Setosa']))

In [ ]:
# 对新样本进行预测
# 测试样本: 花瓣长度=2cm, 花瓣宽度=0.5cm
X_new = np.array([[2.0, 0.5]])
y_pred_new = perceptron.predict(X_new)

class_name = 'Iris-setosa' if y_pred_new[0] == 1 else 'Non-setosa'
print(f"新样本预测结果: {y_pred_new[0]} ({class_name})")

## 6. 可视化决策边界

In [ ]:
def plot_decision_boundary(clf, X, y, title='Decision Boundary'):
    """
    绘制分类器的决策边界
    
    Parameters:
    -----------
    clf : 分类器对象
    X : 特征矩阵
    y : 标签向量
    title : 图表标题
    """
    # 创建网格点
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 200),
        np.linspace(y_min, y_max, 200)
    )
    
    # 对网格点进行预测
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # 绘制决策边界
    plt.figure(figsize=(10, 7))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    plt.contour(xx, yy, Z, colors='black', linewidths=0.5)
    
    # 绘制训练样本
    plt.scatter(X[y == 1, 0], X[y == 1, 1], 
                c='blue', marker='o', label='Iris-setosa', edgecolors='black', s=50)
    plt.scatter(X[y == 0, 0], X[y == 0, 1], 
                c='red', marker='s', label='Non-setosa', edgecolors='black', s=50)
    
    plt.xlabel('Petal Length (cm)', fontsize=12)
    plt.ylabel('Petal Width (cm)', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(loc='upper left', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 绘制决策边界
plot_decision_boundary(perceptron, X, y, 'Perceptron Decision Boundary')

## 7. 感知机的局限性

### 感知机收敛定理

如果训练数据**线性可分**，感知机算法保证能够收敛到一个正确分类所有样本的解。

### 主要局限

1. **无法解决非线性可分问题**: 经典例子是XOR问题
2. **单层结构限制表达能力**: 只能学习线性决策边界
3. **没有概率输出**: 只能给出硬分类结果

### 历史意义

感知机是深度学习的先驱，其局限性促使研究者发展出多层感知机(MLP)和反向传播算法。

In [ ]:
# 演示XOR问题 - 感知机无法解决
print("XOR问题演示")
print("="*40)

# XOR真值表
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])  # XOR运算结果

# 尝试用感知机学习XOR
perceptron_xor = Perceptron(max_iter=1000, random_state=RANDOM_SEED)
perceptron_xor.fit(X_xor, y_xor)

y_pred_xor = perceptron_xor.predict(X_xor)
print("输入    期望输出  预测输出")
for i in range(len(X_xor)):
    print(f"{X_xor[i]}      {y_xor[i]}         {y_pred_xor[i]}")

accuracy_xor = accuracy_score(y_xor, y_pred_xor)
print(f"\nXOR问题准确率: {accuracy_xor:.2f}")
print("结论: 感知机无法正确学习XOR函数，因为XOR不是线性可分的")

## 小结

本教程介绍了:

1. **TLU的基本原理**: 加权求和 + 阶跃函数
2. **感知机训练**: 使用scikit-learn的Perceptron类
3. **二分类任务**: Iris数据集上的Setosa分类
4. **决策边界可视化**: 理解线性分类器的工作方式
5. **感知机局限性**: XOR问题展示了单层网络的局限

下一步可以学习多层感知机(MLP)来解决非线性分类问题。